# Exercise 1
Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd
import numpy as np

In [5]:
programming_languages = ['Python','Java','JavaScript','Html','C','R','C++','C#','PHP']
programming_languages_df=pd.DataFrame(programming_languages,columns=['language'])
programming_languages_df

,language
0,Python
1,Java
2,JavaScript
3,Html
4,C
5,R
6,C++
7,C#
8,PHP


In [8]:
df = spark.createDataFrame(programming_languages_df)
df

DataFrame[language: string]

In [9]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|JavaScript|
|      Html|
|         C|
+----------+
only showing top 5 rows



In [42]:
df.printSchema()

root
 |-- language: string (nullable = true)



# Exercise 2
Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:  
`The 1999 audi a4 has a 4 cylinder engine.` for each vehicle.

In [12]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import col, expr, lit, concat

In [227]:
mpg.select(concat(lit('The '),
                  mpg.year, lit(' '),
                  mpg.manufacturer, lit(' '),
                  mpg.model ,lit(" has a "),
                  mpg.cyl,lit(' cylinder engine')).alias("description"),).show(5,truncate = False)

+----------------------------------------+
|description                             |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



- Transform the trans column so that it only contains either manual or auto.

In [54]:
from pyspark.sql.functions import regexp_extract, regexp_replace
mpg.select(
    'trans',
    regexp_extract("trans", r"^(\w+)", 1).alias("transformation")
).show(5)

+----------+--------------+
|     trans|transformation|
+----------+--------------+
|  auto(l5)|          auto|
|manual(m5)|        manual|
|manual(m6)|        manual|
|  auto(av)|          auto|
|  auto(l5)|          auto|
+----------+--------------+
only showing top 5 rows



# Exercise 3
Load the tips dataset as a spark dataframe._

In [55]:
tips= spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



- What percentage of observations are smokers?

In [232]:
from pyspark.sql.functions import *

In [84]:
tips.filter(tips.smoker =='Yes').count()/(tips.count())

0.38114754098360654

- Create a column that contains the tip percentage

In [234]:
tips.withColumn('tip_percentatge %',
    round((tips.tip /tips.total_bill *100 ),0)
).show(5)

+----------+----+------+------+---+------+----+-----------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentatge %|
+----------+----+------+------+---+------+----+-----------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|              6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|             16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|             14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|             15.0|
+----------+----+------+------+---+------+----+-----------------+
only showing top 5 rows



In [91]:
tips.select(
    (tips.tip /tips.total_bill *100 ).alias("tip_percentage")
).show(5)

+------------------+
|    tip_percentage|
+------------------+
|5.9446733372572105|
|16.054158607350097|
|16.658733936220845|
| 13.97804054054054|
|14.680764538430255|
+------------------+
only showing top 5 rows



- Calculate the average tip percentage for each combination of sex and smoker.

In [98]:
tips.groupBy("sex", "smoker").agg(avg((tips.tip /tips.total_bill *100 ))).show()

+------+------+-------------------------------+
|   sex|smoker|avg(((tip / total_bill) * 100))|
+------+------+-------------------------------+
|  Male|    No|              16.06687151291298|
|  Male|   Yes|             15.277117520248513|
|Female|    No|              15.69209707691836|
|Female|   Yes|              18.21503526994103|
+------+------+-------------------------------+



In [249]:
tip_percentage = (tips.tip /tips.total_bill *100 )
tips.groupby('sex').pivot('smoker').agg(avg(tip_percentage)).show()

+------+-----------------+------------------+
|   sex|               No|               Yes|
+------+-----------------+------------------+
|Female|15.69209707691836| 18.21503526994103|
|  Male|16.06687151291298|15.277117520248513|
+------+-----------------+------------------+



# Exercise 4
Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [107]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Convert the temperatures to farenheight.

In [108]:
weather = weather.withColumn('temp_min',
 expr("ROUND(temp_min * 9/5 +32) ")
)
weather = weather.withColumn('temp_max',
 expr("ROUND(temp_max * 9/5 +32) ")
)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    55.0|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|    51.0|    37.0| 4.5|   rain|
|2012-01-03|          0.8|    53.0|    45.0| 2.3|   rain|
|2012-01-04|         20.3|    54.0|    42.0| 4.7|   rain|
|2012-01-05|          1.3|    48.0|    37.0| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



- Which month has the most rain, on average?

In [111]:
from pyspark.sql.functions import month, year, quarter

(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(avg("precipitation").alias("avg_rainfall"))
    .sort("avg_rainfall")
    .show()
)

+-----+-------------------+
|month|       avg_rainfall|
+-----+-------------------+
|    7|0.38870967741935486|
|    6| 1.1075000000000002|
|    8| 1.3201612903225806|
|    5| 1.6733870967741935|
|    9| 1.9624999999999997|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|    1| 3.7580645161290316|
|   10|  4.059677419354839|
|    3|  4.888709677419355|
|   12|  5.021774193548389|
|   11|  5.354166666666667|
+-----+-------------------+



- Which year was the windiest?

In [112]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(avg("wind").alias("avg_wind"))
    .sort("avg_wind")
    .show()
)

+----+------------------+
|year|          avg_wind|
+----+------------------+
|2013|3.0158904109589058|
|2015| 3.159726027397261|
|2014| 3.387671232876714|
|2012| 3.400819672131148|
+----+------------------+



- What is the most frequent type of weather in January?

In [119]:
(
    weather.filter(month("date") == 1)
    .groupBy("weather")
    .agg(count("weather").alias("days"))
    .show()
)

+-------+----+
|weather|days|
+-------+----+
|    fog|  38|
|drizzle|  10|
|   rain|  35|
|    sun|  33|
|   snow|   8|
+-------+----+



- What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [143]:
(
    weather.filter(month("date") == 7)
    .filter(year("date").isin(['2013','2014']))
    .filter(weather.weather == 'sun')
    .agg(avg("temp_max"), avg('temp_min'))
    .show()
)

+-----------------+-----------------+
|    avg(temp_max)|    avg(temp_min)|
+-----------------+-----------------+
|80.28846153846153|57.53846153846154|
+-----------------+-----------------+



- What percentage of days were rainy in q3 of 2015?

In [ ]:
tips.filter(tips.smoker =='Yes').count()/(tips.count())

In [216]:
number_of_days_in_q3 =(
    weather.filter(quarter("date") == 3)
    .filter(year("date").isin(['2015']))
    .count()
)

In [226]:
(
    weather.filter(quarter("date") == 3)
    .filter(year("date").isin(['2015']))
    .filter(weather.weather =='rain')
    .agg(count('weather')/number_of_days_in_q3)
    .show()
)

+---------------------+
|(count(weather) / 92)|
+---------------------+
| 0.021739130434782608|
+---------------------+



- For each year, find what percentage of days it rained (had non-zero precipitation).

In [251]:
(
    weather.withColumn("year", year("date"))
    .filter(weather.weather =='rain')
    .groupBy("year","weather")
    .agg((count("weather")/365).alias('percentage_of_days'))
    .show()
)

+----+-------+-------------------+
|year|weather| percentage_of_days|
+----+-------+-------------------+
|2013|   rain| 0.1643835616438356|
|2015|   rain| 0.0136986301369863|
|2012|   rain| 0.5232876712328767|
|2014|   rain|0.00821917808219178|
+----+-------+-------------------+

